In [1]:
from dotenv import load_dotenv
import os
from groq import Groq


load_dotenv()

True

In [2]:
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

chat_completion = client.chat.completions.create(
    messages=[{"role": "user", "content": "What is an AI Agent?"}],
    model="llama3-8b-8192",
    temperature=0,
)

print(chat_completion.choices[0].message.content)

An AI agent is a software program that is designed to perform a specific task or set of tasks, and is capable of making decisions and taking actions based on its programming and the data it receives. AI agents can be thought of as autonomous entities that interact with their environment, other agents, and humans to achieve their goals.

There are several key characteristics that define an AI agent:

1. **Autonomy**: AI agents can operate independently, making decisions and taking actions without direct human intervention.
2. **Perception**: AI agents can perceive their environment through sensors, such as cameras, microphones, or other devices.
3. **Action**: AI agents can take actions in their environment, such as moving a robot arm or sending a message.
4. **Reasoning**: AI agents can reason about their environment and make decisions based on their programming and the data they receive.
5. **Learning**: AI agents can learn from their experiences and adapt to new situations.

There ar

In [3]:
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
            result = self.execute()
            self.messages.append({"role": "assistant", "content": result})
            return result
        else:
            print("NO MESSAGE")

    def execute(self):
        completion = client.chat.completions.create(
            model="llama3-70b-8192", messages=self.messages
        )
        return completion.choices[0].message.content

In [4]:
system_prompt = """
You run in a loop of THOUGHT, ACTION, OBSERVATION.
When you have a THOUGHT you return the THOUGHT and ACTION
At the end of the loop you output an ANSWER
Use THOUGHT to describe your THOUGHTS about the question you have been asked.
Use ACTION to run one of the ACTIONS available to you - then return PAUSE.
OBSERVATION will be the result of running those ACTIONS.

Your available ACTIONS are:

# calculate_total:
if amount = 200
then calculate_total(amount)

This returns amount * 1.2

Runs the calculate_total function and returns a JSON FORMAT output as follows:
{"result": 240, "fn": "calculate_total", "next": "PAUSE"}

# get_product_price:

This uses the get_product_price function and passes in the value of the product
e.g. get_product_price('bike')

This uses the get_product_price with a product = 'bike', finds the price of the bike and then returns a JSON FORMAT output as follows:
{"result": 200, "fn": "get_product_price", "next": "PAUSE"}

Example session:

Question: What is total cost of a bike including VAT?
THOUGHT: I need to find the cost of a bike|ACTION|get_product_price|bike

You will be called again with this:

OBSERVATION|200

THOUGHT: I need to calculate the total including the VAT|ACTION|calculate_total|200

You will be called again with this: 

OBSERVATION|240

If you have the ANSWER, output it as the ANSWER.

ANSWER|The price of the bike including VAT is 240

Now it's your turn:
""".strip()


def calculate_total(amount):
    amount = int(amount)
    return int(amount * 1.2)


def get_product_price(product):
    if product == "bike":
        return 400
    if product == "tv":
        return 500
    if product == "laptop":
        return 600

    return None

In [5]:
def loop(max_iterations=10, query: str = ""):

    agent = Agent(client=client, system=system_prompt)

    tools = ["calculate_total", "get_product_price"]

    next_prompt = query

    i = 0

    while i < max_iterations:

        i += 1

        result = agent(next_prompt)

        if "ACTION" in result:
            next = result.split("|")
            print(next)
            next_function = next[2].strip()
            next_arg = str(next[3]).strip()
            # print(next_function, next_arg)

            if next_function in tools:
                result_tool = eval(f"{next_function}('{next_arg}')")
                next_prompt = f"OBSERVATION: {result_tool}"
                print(next_prompt)
            else:
                next_prompt = "OBSERVATION: Tool not found"

            # print(next_prompt)
            continue

        if "ANSWER" in result:
            print(f"Answer found:\n\t{result}")
            break


loop(query="What is cost of a tv including VAT?")

['THOUGHT: I need to find the cost of a tv', 'ACTION', 'get_product_price', 'tv']
OBSERVATION: 500
['THOUGHT: I need to calculate the total including the VAT', 'ACTION', 'calculate_total', '500']
OBSERVATION: 600
Answer found:
	ANSWER|The price of the tv including VAT is 600
